In [63]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
import cpi
import ast

In [109]:
tmdb = pd.read_csv('../data/tmdb_movies.csv',index_col=0)

In [91]:
tmdb.columns = tmdb.columns.str.replace(' ', '').str.lower()

In [92]:
tmdb = tmdb.dropna(subset=['imdb_id'])

In [93]:
print(tmdb['imdb_id'].isna().sum())

0


In [94]:
tmdb = tmdb[(tmdb['revenue'] != 0) & (tmdb['budget'] != 0)]

In [95]:
tmdb['rating'] = tmdb['rating'].round(2)

In [96]:
tmdb = tmdb.sort_values('year', ascending=True)

In [97]:
tmdb.reset_index(drop=True, inplace=True)

In [98]:
cpi.update()


def adjust_for_inflation(row):
    year = row['year']
    budget = row['budget']
    if year <= 2021:
        # Use cpi.inflate for years up to 2021
        return cpi.inflate(budget, year)
    else:
        # Manually adjust for years 2022 and onwards
        if year == 2022:
            # Cumulative inflation for 2021 and 2022
            return budget * 1.07 * 1.065
        elif year == 2023:
            # Cumulative inflation for 2021, 2022 and 2023
            return budget * 1.07 * 1.065 * 1.03
        # Add similar conditions for future years as and when the inflation rate for those years is available
        else:
            return revenue

tmdb['inflation budget'] = tmdb.apply(adjust_for_inflation, axis=1)

tmdb['inflation budget'] = tmdb['inflation budget'].round(0).astype(int)


In [100]:
tmdb = tmdb.rename(columns={'productioncompanies': 'production_companies','inflation budget':'inflation_budget'})


In [101]:
tmdb = tmdb[['imdb_id', 'year', 'title','genres', 'production_companies', 'director','cast', 'budget','inflation_budget','revenue','rating']]


In [107]:
df['genres'] = df['genres'].apply(lambda x: x.strip("[]").replace("'", "").split(', '))
df[['Genre1', 'Genre2']] = pd.DataFrame(df.genres.values.tolist(), index= df.index)

AttributeError: 'int' object has no attribute 'strip'

,imdb_id,year,title,genres,production_companies,director,cast,budget,revenue,rating
0,tt0011549,1920,Over the Hill to the Poorhouse,"['Drama', 'Crime']",Fox Film Corporation,Harry F. Millarde,"['James Sheridan', 'Noel Tearle', 'William Wel...",100000,3000000,0.00
1,tt0010323,1920,The Cabinet of Dr. Caligari,"['Drama', 'Horror', 'Thriller', 'Crime']",Decla Film Gesellschaft Holz & Co.,Robert Wiene,"['Werner Krauß', 'Conrad Veidt', 'Friedrich Fe...",18000,8811,8.00
2,tt0012349,1921,The Kid,"['Comedy', 'Drama']",Charles Chaplin Productions,Charlie Chaplin,"['Charlie Chaplin', 'Jackie Coogan', 'Edna Pur...",250000,2500000,8.20
3,tt0012181,1921,Fool's Paradise,['Romance'],Famous Players-Lasky Corporation,Cecil B. DeMille,"['Dorothy Dalton', 'Conrad Nagel', 'Mildred Ha...",291367,901937,5.50
4,tt0013140,1922,Foolish Wives,"['Drama', 'Thriller']",Universal Film Manufacturing Company,Erich von Stroheim,"['Erich von Stroheim', 'Rudolph Christians', '...",1100000,400200,6.70
...,...,...,...,...,...,...,...,...,...,...
4790,tt5884796,2023,Plane,"['Action', 'Adventure', 'Thriller']",Riverstone Pictures,Jean-François Richet,"['Gerard Butler', 'Mike Colter', 'Yoson An', '...",25000000,51000000,6.89
4791,tt0439572,2023,The Flash,"['Science Fiction', 'Action', 'Adventure']",Warner Bros. Pictures,Andy Muschietti,"['Ezra Miller', 'Sasha Calle', 'Michael Keaton...",190000000,245000000,6.69
4791,tt0439572,2023,The Flash,"['Science Fiction', 'Action', 'Adventure']",Double Dream,Andy Muschietti,"['Ezra Miller', 'Sasha Calle', 'Michael Keaton...",190000000,245000000,6.69
4791,tt0439572,2023,The Flash,"['Science Fiction', 'Action', 'Adventure']",The Disco Factory,Andy Muschietti,"['Ezra Miller', 'Sasha Calle', 'Michael Keaton...",190000000,245000000,6.69
